In [1]:
import numpy as np
import scipy as sci
import scipy.constants as scicon
import matplotlib.pyplot as plt
import pxr_qed

print("pxr_qed is compiled in {:s} precision with {:s} units".
     format(pxr_qed.PRECISION, pxr_qed.UNITS))
if(pxr_qed.HAS_OPENMP):
    print("pxr_qed has openMP support!")
else:
    print("pxr_qed does not have openMP support!")

#Warning: this notebook is conceived for SI units!!

pxr_qed is compiled in double precision with SI units
pxr_qed has openMP support!


In [2]:
m_e = scicon.electron_mass
q_e = scicon.elementary_charge
hbar = scicon.hbar
c = scicon.c
um = scicon.micron
fs = scicon.femto

E_s = m_e**2 * c**3 / (hbar * q_e)
B_s = E_s/c

In [3]:
#Chi functions

chi_how_many = 10
chi_gamma = 10

chi_Ex = (np.random.rand(chi_how_many)-0.5)*2*E_s
chi_Ey = (np.random.rand(chi_how_many)-0.5)*2*E_s
chi_Ez = (np.random.rand(chi_how_many)-0.5)*2*E_s
chi_Bx = (np.random.rand(chi_how_many)-0.5)*2*B_s
chi_By = (np.random.rand(chi_how_many)-0.5)*2*B_s
chi_Bz = (np.random.rand(chi_how_many)-0.5)*2*B_s
chi_px = (np.random.rand(chi_how_many)-0.5)*2*chi_gamma*m_e*c;
chi_py = (np.random.rand(chi_how_many)-0.5)*2*chi_gamma*m_e*c;
chi_pz = (np.random.rand(chi_how_many)-0.5)*2*chi_gamma*m_e*c;

print("chi_phot: ", pxr_qed.chi_photon(
    chi_px, chi_py, chi_pz,
    chi_Ex, chi_Ey, chi_Ez,
    chi_Bx, chi_By, chi_Bz, 1))

print("chi_ele_pos: ", pxr_qed.chi_ele_pos(
    chi_px, chi_py, chi_pz, 
    chi_Ex, chi_Ey, chi_Ez,
    chi_Bx, chi_By, chi_Bz, 1))

chi_phot:  [ 8.214032    4.13900512 20.13433013 10.21405359  2.38524389  7.706629
 13.57250546 11.25000451 12.58074083  8.28222466]
chi_ele_pos:  [ 8.28263056  4.23004793 20.16367636 10.2501489   2.39825269  7.83044641
 13.59139052 11.29990856 12.63852155  8.30455918]


In [4]:
#Breit-Wheeler pair production

bw_dndt_params = pxr_qed.bw.dndt_lookup_table_params()
bw_dndt_params.chi_phot_min = 0.01
bw_dndt_params.chi_phot_max = 100
bw_dndt_params.chi_phot_how_many = 128
print(bw_dndt_params)

bw_dndt_lookup_table = pxr_qed.bw.dndt_lookup_table(bw_dndt_params)
print(bw_dndt_lookup_table)
bw_dndt_lookup_table.generate()
print(bw_dndt_lookup_table)

bw_dndt_lookup_table.save_as("bw_dndt.bin")
bw_dndt_lookup_table_2 = pxr_qed.bw.dndt_lookup_table()
print(bw_dndt_lookup_table_2)
bw_dndt_lookup_table_2.load_from("bw_dndt.bin");
print(bw_dndt_lookup_table_2)

bw_pair_prod_params = pxr_qed.bw.pair_prod_lookup_table_params()
bw_pair_prod_params.chi_phot_min = 0.01
bw_pair_prod_params.chi_phot_max = 100
bw_pair_prod_params.chi_phot_how_many = 64
bw_pair_prod_params.frac_how_many = 64
print(bw_pair_prod_params)

bw_pair_prod_lookup_table = pxr_qed.bw.pair_prod_lookup_table(bw_pair_prod_params)
print(bw_pair_prod_lookup_table)
bw_pair_prod_lookup_table.generate()
print(bw_pair_prod_lookup_table)

bw_pair_prod_lookup_table.save_as("bw_pairprod.bin")
bw_pair_prod_lookup_table_2 = pxr_qed.bw.pair_prod_lookup_table()
print(bw_pair_prod_lookup_table_2)
bw_pair_prod_lookup_table_2.load_from("bw_pairprod.bin");
print(bw_pair_prod_lookup_table_2)

bw_how_many = 10
bw_gamma = 10
bw_dt = 1e-2*fs

bw_Ex = (np.random.rand(bw_how_many)-0.5)*2*E_s
bw_Ey = (np.random.rand(bw_how_many)-0.5)*2*E_s
bw_Ez = (np.random.rand(bw_how_many)-0.5)*2*E_s
bw_Bx = (np.random.rand(bw_how_many)-0.5)*2*B_s
bw_By = (np.random.rand(bw_how_many)-0.5)*2*B_s
bw_Bz = (np.random.rand(bw_how_many)-0.5)*2*B_s
bw_px = (np.random.rand(bw_how_many)-0.5)*2*bw_gamma*m_e*c;
bw_py = (np.random.rand(bw_how_many)-0.5)*2*bw_gamma*m_e*c;
bw_pz = (np.random.rand(bw_how_many)-0.5)*2*bw_gamma*m_e*c
bw_rand = np.random.rand(bw_how_many)
bw_ee = np.sqrt(bw_px**2 + bw_py**2 + bw_pz**2)*c;


bw_chi =  pxr_qed.chi_photon(bw_px, bw_py, bw_pz, bw_Ex, bw_Ey, bw_Ez, bw_Bx, bw_By, bw_Bz)
print("Chi parameters: ", bw_chi)

bw_interp_dndt = bw_dndt_lookup_table.interp(bw_chi)
print("dN/dt lookup table interpolation: ", bw_interp_dndt)

bw_interp_pair_prod = bw_pair_prod_lookup_table.interp(bw_chi, bw_rand)
print("Pair production lookup table interpolation: ", bw_interp_pair_prod)

bw_opt = pxr_qed.bw.get_optical_depth(bw_rand)
print("Breit-Wheeler optical depth: ", bw_opt)

bw_dndt = pxr_qed.bw.get_dn_dt(bw_ee, bw_chi, bw_dndt_lookup_table)
print("Breit-Wheeler dNdt: ", bw_dndt)

print("Breit-Wheeler opt depth before evolution: ", bw_opt)
pxr_qed.bw.evolve_optical_depth(bw_ee, bw_chi, bw_dt, bw_opt, bw_dndt_lookup_table)
print("Breit-Wheeler opt depth after evolution: ", bw_opt)

bw.dndt_lookup_table_params:
	chi_phot_min     : 0.01
	chi_phot_max     : 100
	chi_phot_how_many: 128
bw.dndt_lookup_table:
	is initialized? : False

bw.dndt_lookup_table:
	is initialized? : True

bw.dndt_lookup_table:
	is initialized? : False

bw.dndt_lookup_table:
	is initialized? : True

bw.pair_prod_lookup_table_params:
	chi_phot_min     : 0.01
	chi_phot_max     : 100
	chi_phot_how_many: 64
	frac_how_many    : 64
bw.pair_prod_lookup_table:
	is initialized? : False

bw.pair_prod_lookup_table:
	is initialized? : True

bw.pair_prod_lookup_table:
	is initialized? : False

bw.pair_prod_lookup_table:
	is initialized? : True

Chi parameters:  [ 8.23763417  4.7667929   3.21239066  3.22275238 14.83535786  3.71082428
  4.05725499 10.35281947  6.88023468 13.93923384]
dN/dt lookup table interpolation:  [0.10683571 0.09402127 0.07667365 0.0768411  0.10803941 0.08373078
 0.08770745 0.10861922 0.10404483 0.10839793]
Pair production lookup table interpolation:  [2.92460243 3.10913615 0.81493878 2.

In [5]:
# Quantum Synchrotron radiation

qs_dndt_params = pxr_qed.qs.dndt_lookup_table_params()
qs_dndt_params.chi_part_min = 0.01
qs_dndt_params.chi_part_max = 100
qs_dndt_params.chi_part_how_many = 128
print(qs_dndt_params)

qs_dndt_lookup_table = pxr_qed.qs.dndt_lookup_table(qs_dndt_params)
print(qs_dndt_lookup_table)
qs_dndt_lookup_table.generate()
print(qs_dndt_lookup_table)

qs_dndt_lookup_table.save_as("qs_dndt.bin")
qs_dndt_lookup_table_2 = pxr_qed.qs.dndt_lookup_table()
print(qs_dndt_lookup_table_2)
qs_dndt_lookup_table_2.load_from("qs_dndt.bin");
print(qs_dndt_lookup_table_2)

qs_photem_params = pxr_qed.qs.photon_emission_lookup_table_params()
qs_photem_params.chi_part_min = 0.01
qs_photem_params.chi_part_max = 100
qs_photem_params.frac_min = 1e-12
qs_photem_params.chi_part_how_many = 64
qs_photem_params.frac_how_many = 64
print(qs_photem_params)

qs_dndt_lookup_table.save_as("qs_dndt.bin")
qs_dndt_lookup_table_2 = pxr_qed.qs.dndt_lookup_table()
print(qs_dndt_lookup_table_2)
qs_dndt_lookup_table_2.load_from("qs_dndt.bin");
print(qs_dndt_lookup_table_2)

qs_photon_emission_lookup_table = pxr_qed.qs.photon_emission_lookup_table(qs_photem_params)
print(qs_photon_emission_lookup_table)
qs_photon_emission_lookup_table.generate()
print(qs_photon_emission_lookup_table)

qs_how_many = 10
qs_gamma = 10
qs_dt = 1e-2*fs

qs_Ex = (np.random.rand(qs_how_many)-0.5)*2*E_s
qs_Ey = (np.random.rand(qs_how_many)-0.5)*2*E_s
qs_Ez = (np.random.rand(qs_how_many)-0.5)*2*E_s
qs_Bx = (np.random.rand(qs_how_many)-0.5)*2*B_s
qs_By = (np.random.rand(qs_how_many)-0.5)*2*B_s
qs_Bz = (np.random.rand(qs_how_many)-0.5)*2*B_s
qs_px = (np.random.rand(qs_how_many)-0.5)*2*qs_gamma*m_e*c;
qs_py = (np.random.rand(qs_how_many)-0.5)*2*qs_gamma*m_e*c;
qs_pz = (np.random.rand(qs_how_many)-0.5)*2*qs_gamma*m_e*c;
qs_rand = np.random.rand(qs_how_many)
qs_ee = np.sqrt(1 + (qs_px**2 + qs_py**2 + qs_pz**2)/((m_e*c)**2))*m_e*c**2;


qs_chi =  pxr_qed.chi_ele_pos(qs_px, qs_py, qs_pz, qs_Ex, qs_Ey, qs_Ez, qs_Bx, qs_By, qs_Bz)
print("Chi parameters: ", qs_chi)

qs_interp_dndt = qs_dndt_lookup_table.interp(qs_chi)
print("dN/dt lookup table interpolation: ", qs_interp_dndt)

qs_interp_photon_emission = qs_photon_emission_lookup_table.interp(qs_chi, qs_rand)
print("Pair production lookup table interpolation: ", qs_interp_photon_emission)

qs_opt = pxr_qed.qs.get_optical_depth(qs_rand)
print("Quantum synchrotron optical depth: ", qs_opt)

qs_dndt = pxr_qed.qs.get_dn_dt(qs_ee, qs_chi, qs_dndt_lookup_table)
print("Quantum synchrotron dNdt: ", qs_dndt)

print("Quantum synchrotron opt depth before evolution: ", qs_opt)
pxr_qed.qs.evolve_optical_depth(qs_ee, qs_chi, qs_dt, qs_opt, qs_dndt_lookup_table)
print("Quantum synchrotron opt depth after evolution: ", qs_opt)


qs.dndt_lookup_table_params:
	chi_part_min     : 0.01
	chi_part_max     : 100
	chi_part_how_many: 128
qs.dndt_lookup_table:
	is initialized? : False

qs.dndt_lookup_table:
	is initialized? : True

qs.dndt_lookup_table:
	is initialized? : False

qs.dndt_lookup_table:
	is initialized? : True

qs.photon_emission_lookup_table_params:
	chi_part_min     : 0.01
	chi_part_max     : 100
	frac_min         : 1e-12
	chi_part_how_many: 64
	frac_how_many    : 64
qs.dndt_lookup_table:
	is initialized? : False

qs.dndt_lookup_table:
	is initialized? : True

qs.pair_prod_lookup_table:
	is initialized? : False

qs.pair_prod_lookup_table:
	is initialized? : True

Chi parameters:  [ 9.50808156  6.75001895  3.41219291 15.65528084 14.2492049   7.73768125
  9.03112163  4.70410392  9.1142917  16.30740254]
dN/dt lookup table interpolation:  [ 8.83982023  6.87545355  4.11528502 12.66266781 11.83865945  7.60339577
  8.51447783  5.2517814   8.57167992 13.0369465 ]
Pair production lookup table interpolation:  [0.5

In [4]:
#Schwinger pair production

sc_how_many = 10

sc_Ex = (np.random.rand(sc_how_many)-0.5)*2*E_s
sc_Ey = (np.random.rand(sc_how_many)-0.5)*2*E_s
sc_Ez = (np.random.rand(sc_how_many)-0.5)*2*E_s
sc_Bx = (np.random.rand(sc_how_many)-0.5)*2*B_s
sc_By = (np.random.rand(sc_how_many)-0.5)*2*B_s
sc_Bz = (np.random.rand(sc_how_many)-0.5)*2*B_s

sc_volume = um**3
sc_dt = fs

pair_production_rate = pxr_qed.sc.pair_production_rate(sc_Ex, sc_Ey, sc_Ez, sc_Bx, sc_By, sc_Bz, 1)
exp_pair_number = pxr_qed.sc.expected_pair_number(sc_Ex, sc_Ey, sc_Ez, sc_Bx, sc_By, sc_Bz, sc_volume, sc_dt, 1)

print("pair_production_rate:", pair_production_rate)

print("expected pair number:", exp_pair_number)

pair_production_rate: [2.70470981e+15 3.34812007e+16 1.71281983e+17 6.16755370e+16
 3.01220716e+15 1.42690335e+15 2.46783007e-15 1.87872987e-02
 1.02343404e+17 1.27304765e+17]
expected pair number: [3.52015079e+20 4.35754233e+21 2.22921661e+22 8.02700493e+21
 3.92035528e+20 1.85709939e+20 3.21185434e-10 2.44514675e+03
 1.33198841e+22 1.65685785e+22]
